## Setup

To complete the following guide you will need to install the following packages:
- fireworks-ai
- numpy
- pandas
- pronouncing
- requests
- sentence-transformers
- transformers

You will also need:

- Fireworks account (https://fireworks.ai/)
- Fireworks API key
- The firectl command-line interface (https://docs.fireworks.ai/tools-sdks/firectl/firectl)

In [1]:
import json

from fireworks.client import Fireworks
import numpy as np
import pandas as pd
import pronouncing
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
embeddings_model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

/Users/acrobat/.local/share/virtualenvs/fine-tuning-workshop-WXOdYiJS/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/acrobat/.local/share/virtualenvs/fine-tuning-workshop-WXOdYiJS/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details 

In [2]:
# Sign-in to your Fireworks account
!firectl signin

Signed in as: jayozer@gmail.com
Account ID: jayozer-ce1cd6


In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variable
api_key = os.getenv('FIREWORKS_API_KEY')

client = Fireworks(api_key=api_key)

# Replace the line below with your Fireworks account id
account_id = os.getenv('FIREWORKS_ACCOUNT_ID')

## Problem Definition: Poem Generation

*Note: The poem topics used in this example were synthetically generated by Claude 3 Opus*

LLMs are capable of performing creative writing tasks. However, assessing the quality of such tasks, like poetry generation, is highly subjective.

### Task
In last week's notebook, we created a framework to quantitatively evaluate LLM-generated poetry. This week, you'll observe how to further improve this solution.

As I am not a professional poet, I am unable to write high-level poetry myself. The ideal approach would be to search the web for high-quality poems that matches the style I want the LLM to generate. However, this method is very time-consuming. A more efficient approach is to use the "critique and revise" method, where the LLM first generates critiques on how each poem can be improved. We then ask the LLM to rewrite the poems based on these critiques. Finally, we fine-tune the LLM on the revised poems.

### Data
The data can be found in the week-4 data folder.

We will use the following datasets:
- `./data/training_poem_topics.csv`
- `./data/test_poem_topics.csv`

Each of those datasets consists of 100 unique poem topics. 

In [5]:
training_data = pd.read_csv('/Users/acrobat/Documents/GitHub/fine-tuning-workshop/src/week-4/data/training_poem_topics.csv')
test_data = pd.read_csv('/Users/acrobat/Documents/GitHub/fine-tuning-workshop/src/week-4/data/test_poem_topics.csv')

### Foundation Model Baseline
Our first step is to generate a poem for each of the topics in the training data using a foundation_model. We will then use the critique and revise method to improve upon these poems.

In [6]:
# Given a csv file with a list of topics, generates a poem for each topic
system_message = 'You are a professional poet. Write a unique and original contemporary poem about the topic suggested by the user. Your response should contain ONLY the content of the poem.'
def generate_poems(model, df):
    responses = list()
    for i, row in enumerate(df.iterrows()):
        response = client.chat.completions.create(
            model=model,
            messages=[
              {"role": "system", "content": system_message},
              {"role": "user", "content": row[1]['topic']}
            ],
        )
        response = response.choices[0].message.content
        responses.append(response)    
    return responses

In [7]:
# We first generate poems for poetry topics in our training set
llama_70b_training_poems = generate_poems('accounts/fireworks/models/llama-v3p1-70b-instruct', training_data)

### Critique
In the critique step, we create a scoring rubric and ask an LLM to generate improvements to the previously created poems based on the rubric.

In [8]:
# We now use our scoring rubric to generate a list of critiques about each poem
poem_guidelines = """- Is the poem original?
- Does the poem contain beauty, power, education or entertainment?
- is the message of the poem clear? Is it a good message, or is it of little value to anyone?
- Is the poem clear in its expression? Does it maintain coherence throughout?
- If the poem is written in rhyming verse, then it should be rated according to how well the rhymes fit, not only with each other, but with the flow and the intended nuance of meaning the verse demands.
- What form does the poem take? Is it a sonnet, free verse, haiku, etc.? How does the form contribute to the poem's impact?
- Does the poet us the best possible choice of words in the poem? A person can ball, cry, sob, whimper, and shed tears, but which term would best fit the mood the poet is trying to convey?"""

poem_crtique_rubric = f'''You are professional poet responsible for assessing the quality of AI generated poems.

Assessment Guidelines:
{poem_guidelines}

Given the above guidelines, provide a list of ways that the poem could be improved.'''

def critique_poems(poems, evaluation_model):
    critiques = list()
    for poem in poems:
        response = client.chat.completions.create(
            model=evaluation_model,
            messages=[
                {"role": "system", "content": poem_crtique_rubric},
                {"role": "user", "content": poem}
            ],
        )

        try: 
            response = response.choices[0].message.content
            critiques.append(response)
        except json.JSONDecodeError as jde:
            continue

    return critiques

In [9]:
llama_70b_training_critiques = critique_poems(llama_70b_training_poems, 'accounts/fireworks/models/llama-v3p1-70b-instruct')

In [10]:
llama_70b_training_critiques[3]

'What a beautiful and imaginative poem! Based on the assessment guidelines, here are some suggestions for improvement:\n\n1. **Originality**: While the poem explores a unique and fascinating concept, some of the language and imagery may feel a bit familiar (e.g., "skies of sapphire and amethyst," "cities floated," "trees like sentinels of silver and gold"). Consider adding more distinct and unexpected elements to make the poem truly stand out.\n2. **Clarity and coherence**: The poem\'s structure and language are generally clear, but some stanzas feel a bit disconnected from each other. For example, the transition from the third stanza to the fourth feels a bit abrupt. Consider revising the poem to create a more cohesive narrative flow.\n3. **Message and value**: The poem\'s message is largely one of nostalgia and longing for a lost world. While this is a powerful and relatable theme, it may benefit from a clearer sense of purpose or resolution. Consider adding more depth or nuance to t

### Revise
In the revise step, we create a new prompt that tells the LLM to generate a revised poem, given the previously generated critiques.

In [11]:
# We now give the LLM both the poem and the critiques, and tell it to improve the poem based on the following critiques.
improvement_sys_message = '''You are a professional poet. Improve the poem, given the following critiques.

Your response must ONLY contain the content of the improved poem. DO NOT TELL ME YOUR CHANGES, JUST GIVE ME THE REVISED POEM!'''

def generate_improved_poems(model, poems, critiques):
    responses = list()
    for i, poem in enumerate(poems):

        user_message = f''''
poem:      
{poem}

critiques:
{critiques[i]}'''
        
        response = client.chat.completions.create(
            model=model,
            messages=[
              {"role": "system", "content": improvement_sys_message},
              {"role": "user", "content": user_message}
            ],
        )
        response = response.choices[0].message.content
        responses.append(response)    

    return responses

In [12]:
llama_70b_training_improved_poems = generate_improved_poems('accounts/fireworks/models/llama-v3p1-70b-instruct', llama_70b_training_poems, llama_70b_training_critiques)

In [14]:
print(llama_70b_training_improved_poems[3])

In the hollow of my mind, a creaking door
A murmured vow of a world beyond the astral shore
A dimension shrouded, yet echoes seep
A yearning to return, a hollowed-out ache

I recall the firmament of cerulean and heliotrope
Where cities drifted, and rivers flowed in auroral mist
The trees, like silvan guardians of silver and orichalc
Their leaves, a susurrus of cryptic whispers, a language unspun

In this forgotten realm, I wandered, unmoored
A stranger in a world of wonder, where the impossible roamed
The creatures of this land, a cacophony of forms
Dancing on the solar wind, a kaleidoscope of chromatic norms

The memories, a palimpsest of fractured time
A puzzle I've tried to reassemble, in the syntax of the mind
But like the shards of a shattered prism's refracted light
Reflections distorted, a reality unseen, yet felt in the night

In dreams, I still revisit this mystic littoral
Where the moon dipped in the ocean, and the sun rose, a burning coal
But when I wake, the recollections d

### Fine-Tuning
We know fine-tune a smaller LLM using the revised poems. This is similar to the knowledge distillation method from last week's notebook, except we are fine-tuning on the revised poems of the larger model, rather than the original poems that it generated.

In [15]:
# Upload the improved poems to fireworks as our fine-tuning dataset
def format_poem_for_fireworks(topic, poem):
    return {"messages": [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": topic}, 
        {"role": "assistant", "content": poem}
    ]}

topics = training_data['topic'].tolist()
json_objs = list()
for i, poem in enumerate(llama_70b_training_improved_poems):
    msg = {"messages": [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": topics[i]}, 
        {"role": "assistant", "content": poem}
    ]}    
    json_objs.append(msg)

dataset_file_name = 'poem_training_data.jsonl'
dataset_id = 'improved-poem-data-v1'
with open(dataset_file_name, 'w') as f:
    for obj in json_objs:
        json.dump(obj, f)
        f.write('\n')

In [16]:
# Upload our dataset to fireworks
!firectl create dataset {dataset_id} {dataset_file_name}

178.55 KiB / 178.55 KiB [---------------------------] 100.00% 5.13 MiB p/s 200ms


In [20]:
!pwd

/Users/acrobat/Documents/GitHub/fine-tuning-workshop/src/week-4


In [26]:
# Create a fine-tuning job
!firectl create fine-tuning-job --settings-file poem_generation_fine_tuning_config.yaml --display-name improved-poems-v1 --dataset {dataset_id} 

Name: accounts/jayozer-ce1cd6/fineTuningJobs/b203ef3333a04c1d95cd37e7ab695b71
Display Name: improved-poems-v1
Create Time: 2024-09-25 14:49:32
State: CREATING
Dataset: accounts/jayozer-ce1cd6/datasets/improved-poem-data-v1
Status: OK
Created By: jayozer@gmail.com
Conversation:
  Jinja Template: 
{%- set _mode = mode | default('generate', true) -%}
{%- set stop_token = '<|eot_id|>' -%}
{%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}
{%- set ns = namespace(initial_system_message_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}
{%- for message in ns.messages -%}
    {%- if not message.get('role') -%}
        {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}
    {%- endif -%}
    {%- if message['role'] | upper not in message_roles -%}
        {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}
    {%- endif -%}
    {%- if 'content' not in message -%}
       

In [27]:
# NOTE THAT THIS ID WILL CHANGE WHEN YOU RUN THE FINE-TUNING JOB ON YOUR ACCOUNT!!!
# The model id is printed in the stdout of the cell above as Name: accounts/{account_id}/fineTuningJobs/{ft_model_id}
ft_model_id = 'b203ef3333a04c1d95cd37e7ab695b71' 

In [29]:
# Wait until the State of the fine-tuning job is listed as COMPLETED (~10-20 minutes)
!firectl get fine-tuning-job {ft_model_id}

Name: accounts/jayozer-ce1cd6/fineTuningJobs/b203ef3333a04c1d95cd37e7ab695b71
Display Name: improved-poems-v1
Create Time: 2024-09-25 14:49:32
State: RUNNING
Dataset: accounts/jayozer-ce1cd6/datasets/improved-poem-data-v1
Status: OK
Created By: jayozer@gmail.com
Conversation:
  Jinja Template: 
{%- set _mode = mode | default('generate', true) -%}
{%- set stop_token = '<|eot_id|>' -%}
{%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}
{%- set ns = namespace(initial_system_message_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}
{%- for message in ns.messages -%}
    {%- if not message.get('role') -%}
        {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}
    {%- endif -%}
    {%- if message['role'] | upper not in message_roles -%}
        {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}
    {%- endif -%}
    {%- if 'content' not in message -%}
        

### Evaluation
Finally, we evaluate the fine-tuned model on our test data. In the previous weeks notebook, the knowledge distillation method resulted in an average LLM judge score of 8.21. We expect to receive a higher score now that we are fine-tuning on the revised poems rather than the initial poems that the large model generated.

In [30]:
# Deploy the fine-tuned model
!firectl deploy {ft_model_id}

In [31]:
# Wait until the the Deploymed Model Refs lists the state of the model as "DEPLOYED" (~5-20 minutes).
!firectl get model {ft_model_id}

Name: accounts/jayozer-ce1cd6/models/b203ef3333a04c1d95cd37e7ab695b71
Create Time: 2024-09-25 15:12:55
State: READY
Status: OK
Kind: HF_PEFT_ADDON
Base Model Details:
  Checkpoint Format: CHECKPOINT_FORMAT_UNSPECIFIED
Peft Details:
  Base Model: accounts/fireworks/models/llama-v3p1-70b-instruct
  R: 32
  Target Modules: [v_proj, k_proj, q_proj, gate_proj, o_proj, down_proj, up_proj]
Conversation Config:
  Style: jinja
Context Length: 131072
Fine Tuning Job: accounts/jayozer-ce1cd6/fineTuningJobs/b203ef3333a04c1d95cd37e7ab695b71
Deployed Model Refs: 
  [{
    Name: accounts/jayozer-ce1cd6/deployedModels/b203ef3333a04c1d95cd37e7ab695b71-68342157
    Deployment: accounts/fireworks/deployments/5bb3209d
    State: DEPLOYED
    Default: true
  }]


In [32]:
# Generate poems on the test set using our fine-tuned model
ft_poems = generate_poems(f'accounts/{account_id}/models/{ft_model_id}', test_data)

In [48]:
# Evaluate poems based on their average length (# of characters)
def calculate_avg_length(poems):
    return int(np.mean([len(poem) for poem in poems]))

# Evaluate poems based on the pct of stanzas that contain a rhyme
def calculate_rhyming_fct(poem):
    stanzas = poem.split('\n\n')
    stanzas = [stanza for stanza in stanzas if len(stanza.split('\n')) >= 1]
    
    num_rhyming_stanzas = 0
    for stanza in stanzas:
        lines = stanza.split('\n')
        end_words = [line.split(' ')[-1].strip('.?!"\',') for line in lines]
        found_rhyme = False
        for i in range(len(end_words)):
            for j in range(i + 1, len(end_words)):
                found_rhyme = True if found_rhyme or (end_words[j] in pronouncing.rhymes(end_words[i])) else False
                
        if found_rhyme:
            num_rhyming_stanzas += 1

    if not len(stanzas):
        print(poem)
    return num_rhyming_stanzas / len(stanzas)

# Evaluate poems based on how often they have a positive sentiment
def has_positive_sentiment(poem):
    try:
        sentiment = sentiment_pipeline(poem)[0]
        return True if sentiment['label'] == 'POSITIVE' else False
    except:
        return True

In [49]:
# Calculate heuristics of our fine-tuned poems
print("Heuristic Evaluation")
print(f'Average Length: {calculate_avg_length(ft_poems)}')
print(f"Rhyming Pct: {int(100 * np.mean([calculate_rhyming_fct(poem) for poem in ft_poems]))}%")
print(f"Positive Sentiment: {int(100 * np.mean([has_positive_sentiment(poem) for poem in ft_poems]))}%")

Heuristic Evaluation
Average Length: 1255
Rhyming Pct: 73%
Positive Sentiment: 92%


In [51]:
# Evaluate poems using the LLM as a Judge strategy
poem_evaluation_rubric = f'''You are professional poet responsible for assessing the quality of AI generated poems.

Score each poem on a scale of 0 to 10, where 10 represents the best possible poem.

Scoring Guidelines:
{poem_guidelines}

Think through your reasoning step-by-step and explain your reasoning. Steps for judging a poem:
1. Read the Poem Multiple Times: Read it aloud and silently to capture both the meaning and the sound.
2. Take Notes: Jot down initial impressions, notable phrases, and any questions that arise.
3. Analyze the Elements: Break down the poem into its components (content, structure, language, sound).
4. Reflect on Your Experience: Consider your emotional response and personal connection to the poem.

The last line in your response MUST be a json object {{"score": XXX}}, where XXX is the score you are giving the response.'''

def evaluate_poems(poems, evaluation_model):
    scores = list()
    for poem in poems:
        response = client.chat.completions.create(
            model=evaluation_model,
            messages=[
                {"role": "system", "content": poem_evaluation_rubric},
                {"role": "user", "content": poem}
            ],
            temperature=0,
        )

        try: 
            response = response.choices[0].message.content
            score = int(json.loads(response.split('\n')[-1])['score'])  
            scores.append(score)
        except json.JSONDecodeError as jde:
            continue
        
    return sum(scores) / len(scores)

In [52]:
# Use the LLM to evaluate our fine-tuned model
ft_avg_score = evaluate_poems(ft_poems, 'accounts/fireworks/models/llama-v3p1-70b-instruct')
print(f"Avg LLM Judge Score: {round(ft_avg_score , 2)}")

Avg LLM Judge Score: 8.32


In [33]:
# Undeploy the fine-tuned model (does not cost anything extra, but Fireworks may limit your number of deployed models).
!firectl undeploy {ft_model_id}